# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import requests
import gmaps
import os

#define API key
import configparser
config = configparser.ConfigParser()
initfile = "..\\config\\auth.ini"
config.read(initfile)
g_key = config.get('GoogleAPIKey','g_key')

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("../output_data/cities.csv", encoding='utf-8').drop(['Unnamed: 0'],axis=1)
cities_df

,City,Country,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date
0,vaini,TO,-21.2000,-175.2000,84.20,84,75,5.75,1619144226
1,kapaa,US,22.0752,-159.3190,79.00,65,20,10.36,1619144227
2,provideniya,RU,64.3833,-173.3000,41.00,48,0,11.18,1619144227
3,egvekinot,RU,66.3167,-179.1667,30.16,81,21,1.21,1619144227
4,alofi,NU,-19.0595,-169.9187,84.20,74,40,11.50,1619144228
...,...,...,...,...,...,...,...,...,...
539,ngunguru,NZ,-35.6167,174.5000,66.78,74,100,10.54,1619144309
540,russell,NZ,32.3502,-85.1999,55.40,71,1,2.35,1619144309
541,asau,TV,46.4333,26.4000,42.01,60,83,2.30,1619144309
542,beringovskiy,RU,63.0500,179.3167,29.05,71,99,8.25,1619144309


In [3]:
len((cities_df[cities_df["Latitude"]<-90])|(cities_df[cities_df["Latitude"]>90]))

0

In [5]:
locations = cities_df[['Latitude','Longitude']]
weights = cities_df['Humidity (%)'].astype(float)
max_humidity = np.max(weights)
print(weights)
print(f"Maximum humidity among the cities is:{max_humidity}")
locations 

0      84.0
1      65.0
2      48.0
3      81.0
4      74.0
       ... 
539    74.0
540    71.0
541    60.0
542    71.0
543    88.0
Name: Humidity (%), Length: 544, dtype: float64
Maximum humidity among the cities is:100.0


,Latitude,Longitude
0,-21.2000,-175.2000
1,22.0752,-159.3190
2,64.3833,-173.3000
3,66.3167,-179.1667
4,-19.0595,-169.9187
...,...,...
539,-35.6167,174.5000
540,32.3502,-85.1999
541,46.4333,26.4000
542,63.0500,179.3167


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

# Customize the size of the figure
figure_layout = {
    'width':'1200px',
    'height':'900px',
    'border':'1px solid black',
    'padding':'1px',
    'margin':'0 auto 0 auto'
}
fig = gmaps.figure(layout = figure_layout)
heat_layer = gmaps.heatmap_layer(locations,weights= weights,dissipating=False,max_intensity=max_humidity,point_radius=2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='900px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
#Find the cities with ideal weather
ideal_weather_cities_df = cities_df.loc[(cities_df["Max Temperature (F)"]>70)& (cities_df["Max Temperature (F)"]<80)&(cities_df["Wind Speed (mph)"]<10)&(cities_df["Cloudiness (%)"]==0)].dropna()
ideal_weather_cities_df

,City,Country,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date
112,benjamin constant,BR,-4.3831,-70.0311,77.00,94,0,1.66,1619144243
182,cabedelo,BR,-6.9811,-34.8339,77.00,88,0,2.30,1619144253
192,oussouye,SN,12.4850,-16.5469,73.40,88,0,4.61,1619144255
205,atar,MR,20.5169,-13.0499,75.96,14,0,7.45,1619144257
285,sohag,EG,26.5570,31.6948,71.60,24,0,4.61,1619144268
309,sakakah,SA,29.9697,40.2064,77.00,18,0,6.91,1619144273
340,salalah,OM,17.0151,54.0924,78.80,73,0,3.44,1619144278
451,flinders,AU,-34.5833,150.8552,72.00,41,0,4.00,1619144295
463,fukuma,JP,33.7667,130.4667,75.99,28,0,8.05,1619144297


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
# create hotel_df
hotel_df = ideal_weather_cities_df[["City","Country","Latitude","Longitude"]]
#Add a "Hotel Name" column
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
112,benjamin constant,BR,-4.3831,-70.0311,
182,cabedelo,BR,-6.9811,-34.8339,
192,oussouye,SN,12.4850,-16.5469,
205,atar,MR,20.5169,-13.0499,
285,sohag,EG,26.5570,31.6948,
309,sakakah,SA,29.9697,40.2064,
340,salalah,OM,17.0151,54.0924,
451,flinders,AU,-34.5833,150.8552,
463,fukuma,JP,33.7667,130.4667,


In [10]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "radius" : 5000,  #search for hotels within 5000 meters of city lat & lng
          "key" : g_key}

for index,row in hotel_df.iterrows():
    city = row["City"]
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    params["location"] = f"{lat},{lng}"
    response = requests.get(base_url, params=params).json()

# I realised the code with response['results'][0] doesnt return operational hotels for cities with ideal weather
#     hotel_name = response['results'][0]['name']
#     hotel_df.loc[index,"Hotel Name"] = hotel_name
#     print(f"The hotel close to city {city.title()} is hotel {hotel_name}")

# So I had to use the code below to be able to get operational hotels in cities with ideal weather
    for i in range(0,len(response['results'])):
        try:
            if response['results'][i]["business_status"] == "OPERATIONAL":
                hotel_name = response['results'][i]['name']
                hotel_df.loc[index,"Hotel Name"] = hotel_name
                print(f"The nearest operational hotel to city {city.title()} is hotel {hotel_name}")
                break
            else:
                continue
        except KeyError:
            continue
    
hotel_df

The nearest operational hotel to city Benjamin Constant is hotel Igreja Presbiteriana de Benjamim Constant(I.P.B.C)
The nearest operational hotel to city Cabedelo is hotel IFPB - Campus Cabedelo
The nearest operational hotel to city Oussouye is hotel Asamblea de Cooperacion por la Paz (Senegal)
The nearest operational hotel to city Atar is hotel Oumou Elghoura
The nearest operational hotel to city Sohag is hotel Doctors Club
The nearest operational hotel to city Sakakah is hotel Fakhamat Al Orjoana Apartment
The nearest operational hotel to city Salalah is hotel HAMDAN PLAZA HOTEL SALALAH
The nearest operational hotel to city Flinders is hotel Shellharbour Resort & Conference Centre
The nearest operational hotel to city Fukuma is hotel Hamazushi


,City,Country,Latitude,Longitude,Hotel Name
112,benjamin constant,BR,-4.3831,-70.0311,Igreja Presbiteriana de Benjamim Constant(I.P....
182,cabedelo,BR,-6.9811,-34.8339,IFPB - Campus Cabedelo
192,oussouye,SN,12.4850,-16.5469,Asamblea de Cooperacion por la Paz (Senegal)
205,atar,MR,20.5169,-13.0499,Oumou Elghoura
285,sohag,EG,26.5570,31.6948,Doctors Club
309,sakakah,SA,29.9697,40.2064,Fakhamat Al Orjoana Apartment
340,salalah,OM,17.0151,54.0924,HAMDAN PLAZA HOTEL SALALAH
451,flinders,AU,-34.5833,150.8552,Shellharbour Resort & Conference Centre
463,fukuma,JP,33.7667,130.4667,Hamazushi


In [11]:
# rename Latitude and Longitude columns to Lat and Lng
hotel_df = hotel_df.rename(columns ={"Latitude":"Lat","Longitude":"Lng"})
hotel_df

,City,Country,Lat,Lng,Hotel Name
112,benjamin constant,BR,-4.3831,-70.0311,Igreja Presbiteriana de Benjamim Constant(I.P....
182,cabedelo,BR,-6.9811,-34.8339,IFPB - Campus Cabedelo
192,oussouye,SN,12.4850,-16.5469,Asamblea de Cooperacion por la Paz (Senegal)
205,atar,MR,20.5169,-13.0499,Oumou Elghoura
285,sohag,EG,26.5570,31.6948,Doctors Club
309,sakakah,SA,29.9697,40.2064,Fakhamat Al Orjoana Apartment
340,salalah,OM,17.0151,54.0924,HAMDAN PLAZA HOTEL SALALAH
451,flinders,AU,-34.5833,150.8552,Shellharbour Resort & Conference Centre
463,fukuma,JP,33.7667,130.4667,Hamazushi


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content = hotel_info)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='900px', margin='0 auto 0 auto', padding='1px', wi…